In [ ]:
!curl --request GET --url host.docker.internal:21434/api/tags

In [ ]:
from langchain_ollama import ChatOllama

### llm 
local_llm="deepseek-r1:1.5b-qwen-distill-fp16"
llm = ChatOllama(model=local_llm, temperature=0, base_url="http://host.docker.internal:21434")
llm_json = ChatOllama(model=local_llm, temperature=0, return_json=True, base_url="http://host.docker.internal:21434")

In [ ]:
msg = llm.invoke("What is the capital of France?")
msg.content

__________________

# GROQ test

In [ ]:
import os

from groq import Groq
from dotenv import load_dotenv
load_dotenv("../../groq.env")


In [ ]:
client = Groq(
    # This is the default and can be omitted
    api_key=os.getenv("GROQ_API_KEY"),
)

In [ ]:
model_name = os.getenv("GROQ_LLM")
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "you are a helpful assistant."
        },
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model=model_name,
    reasoning_format={"type": "groq"},
    temperature=0.6,
    max_completion_tokens=131072,
    top_p=0.95,
    stream=True,
    stop=None,
    
)

In [ ]:

for chunk in chat_completion:
    print(chunk.choices[0].delta.content or "", end="")

__________


# GROQ langchain test

In [ ]:
import os
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
from ollama_deep_researcher.prompts import (
    query_writer_instructions,
    summarizer_instructions,
    reflection_instructions,
    get_current_date,
)
from dotenv import load_dotenv
load_dotenv("../../groq.env")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
model_name = os.getenv("GROQ_LLM")

In [ ]:
current_date = get_current_date()
formatted_prompt = query_writer_instructions.format(
        current_date=current_date, research_topic="fast language models"
    )

In [ ]:
llm = ChatGroq(
    model=model_name,
    temperature=0,
    response_format={"type": "json_object"},
)
#llm_json_mode = llm.with_structured_output(method="json_mode")

In [ ]:
result = llm_json_mode.invoke(
    [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content=f"Generate a query for web search:"),
    ]
)

In [ ]:
result2 = llm.invoke(
    [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content=f"Generate a query for web search:"),
    ]
)

In [ ]:
print(result2.content)

_____

# tavily test (Used for making the crawler search)

In [ ]:
import os
from ollama_deep_researcher.utils import (
    deduplicate_and_format_sources,
    tavily_search,
    format_sources,
)
from dotenv import load_dotenv
load_dotenv("../../.env")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [ ]:
search_results = tavily_search("what is the capital of France?", fetch_full_page=True, max_results=3)

In [ ]:
search_results

In [ ]:
search_str = deduplicate_and_format_sources(
            search_results,
            max_tokens_per_source=10000000,
            fetch_full_page=True,
        )

In [ ]:
print(search_str)

_______

# Test rets_info_scraper and translate

In [ ]:
from ollama_deep_researcher.retsinfo_crawl import retsinfo_search_and_crawl, retsinfo_search, extract_ids
from ollama_deep_researcher.translate_async import deduplicate_translate_and_format_sources
from langchain_groq import ChatGroq


In [ ]:
if __name__ == "__main__":
    search_r = await retsinfo_search("Justitsministerens beføjelser til fastsættelse af regler for indsamlinger", max_results=3)

In [ ]:
search_r

In [ ]:
if __name__ == "__main__":
    id_url_pari = await extract_ids(search_r)

In [ ]:
id_url_pari.values()

In [ ]:

if __name__ == "__main__":
    # Load environment variables from .env file
   result = await retsinfo_search_and_crawl(query="Justitsministerens beføjelser til fastsættelse af regler for indsamlinger", max_results=3)
    

In [ ]:

for r in result["results"]:
    print(r['title'])
result["amount"]

In [ ]:
llm_translate_s = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=64000,
    service_tier="auto"
        )

In [ ]:
if __name__ == "__main__":
    search_str_en, saved_first_en = await deduplicate_translate_and_format_sources(result, llm_translate_s, 64000)

In [ ]:
print(search_str_en)

__________

_____

# misc.

test qurry
```sh 
Lav en liste over hvad Justitsministeren kan fastsætte, I den juridiske contexten indsamlinger?
```

In [2]:
array = [1,2]

array.append(3)

array[-2]

2

params = {"dt":[10,1480,20,30,40,50,90,120,270,60,100,80,110,130,140,150,160,170,180,200,210,220,1510,1490,-10,230,240,250,260,980]}

In [15]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from dotenv import load_dotenv
import os
load_dotenv("../../.env")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("OPENAI_MODEL")
base_url = os.getenv("OPENAI_BASE_URL")

In [16]:
llm_translate_q = ChatOpenAI(
    model=model_name,
    base_url=base_url,
    temperature=1,
    max_tokens=12000,
)

In [17]:
result = await llm_translate_q.ainvoke(
    [
        SystemMessage(content="you are a helpful assistant"),
        HumanMessage(content="what is the capital of France?"),
    ]
)